# Applied Data Science Capstone
----------
### Week 3 Peer Graded Assignment

* ## Assignment 1
Scrape wikipedia for the table on neighborhoods in Canada and format it into a pandas DataFrame

In [5]:
#importing needed libraries
import requests
import pandas as pd
from pandas.io.json import json_normalize
!pip install lxml              #uncomment to install lxml parser
!pip install beautifulsoup4    #uncomment to install bs
from bs4 import BeautifulSoup

     |████████████████████████████████| 5.5MB 8.2MB/s eta 0:00:01     |███▋                            | 624kB 8.2MB/s eta 0:00:01
     |████████████████████████████████| 122kB 4.5MB/s eta 0:00:01


### Making url request and grabbing requested table

In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
s = requests.Session()
response = s.get(url, timeout = 10)
soup = BeautifulSoup(response.content, "html.parser")

# to fix the content idents etc and make it readable
pretty_soup = soup.prettify()
# finding the correct table and its headers
table = soup.find('table', {'class' : 'wikitable sortable'})

### Extracting column names and rows from table

In [7]:
rows = table.find_all("tr")
headers = [th.text.rstrip() for th in rows[0].find_all("th")]
codes_in_example = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A']

# renaming Postal Code header to PostalCode, as per course instructions
headers[0] = 'PostalCode'
print(" Table Headers : ", headers)

# grabbing rows
table_rows = []
for row in rows[1:]:
    data = [d.text.rstrip() for d in row.find_all("td")]
    if data[1].upper()=='NOT ASSIGNED':                # Reject rows that have no Borough
        continue
    if data[2].upper()=='NOT ASSIGNED':                # If no neighborhood data exists, copy from Borough data
        data[2] = data[1]
    table_rows.append(data)

 Table Headers :  ['PostalCode', 'Borough', 'Neighbourhood']


### Creating a Pandas DataFrame with the extracted data

In [8]:
df = pd.DataFrame(data = table_rows, columns = headers)
#df.loc[df['PostalCode'] == "M5A"]     #Checking if my data matches data from example in course
df

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Print Specific dataframe from example

In [9]:
df.loc[df['PostalCode'].isin(codes_in_example)]               #Printing specific dataframe from example

,PostalCode,Borough,Neighbourhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M1B,Scarborough,"Malvern, Rouge"
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
23,M4G,East York,Leaside
24,M5G,Downtown Toronto,Central Bay Street
27,M2H,North York,Hillcrest Village
32,M1J,Scarborough,Scarborough Village
50,M9L,North York,Humber Summit
54,M4M,East Toronto,Studio District
71,M1R,Scarborough,"Wexford, Maryvale"


In [10]:
df.shape

(103, 3)

***Assgnment 1 ENDS HERE***


---------------------------------


* ## Assignment 2
Get the latitude and the longitude coordinates of each neighborhood using Geocoder and add to the DataFrame for later use with Foursquare

In [11]:
!pip install geocoder      #uncomment to install Geocoder package
import geocoder

     |████████████████████████████████| 102kB 17.2MB/s ta 0:00:01
     |████████████████████████████████| 829kB 8.1MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


### Fetch location data for each postal code in Toronto, Ontario from geocodefarm or arcgis

In [14]:
latitudes = []
longitudes = []
for postal in df['PostalCode']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal))
    print(postal, ", found coordinates : ", g.latlng)
    while g.latlng == None:
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal))
        print(postal, ", found coordinates : ", g.latlng)
    latlng = g.latlng
    latitudes.append(latlng[0])
    longitudes.append(latlng[1])
print("====================")
print("======Complete======")
print("====================")

M3A , found coordinates :  [43.75245000000007, -79.32990999999998]
M4A , found coordinates :  [43.73057000000006, -79.31305999999995]
M5A , found coordinates :  [43.65512000000007, -79.36263999999994]
M6A , found coordinates :  [43.72327000000007, -79.45041999999995]
M7A , found coordinates :  [43.66253000000006, -79.39187999999996]
M9A , found coordinates :  [43.662630000000036, -79.52830999999998]
M1B , found coordinates :  [43.811390000000074, -79.19661999999994]
M3B , found coordinates :  [43.74923000000007, -79.36185999999998]
M4B , found coordinates :  [43.70718000000005, -79.31191999999999]
M5B , found coordinates :  [43.65739000000008, -79.37803999999994]
M6B , found coordinates :  [43.70687000000004, -79.44811999999996]
M9B , found coordinates :  [43.65034000000003, -79.55361999999997]
M1C , found coordinates :  [43.78574000000003, -79.15874999999994]
M3C , found coordinates :  [43.72168000000005, -79.34351999999996]
M4C , found coordinates :  [43.68970000000007, -79.306819999

### Combine location data with dataframe

In [15]:
df['Latitude'] = latitudes
df['Longitude'] = longitudes
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


In [16]:
df.loc[df['PostalCode']=='M5A']        #Checking if my location data is similar to the one displayed on course example

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264


### Print Specific dataframe from example with location data

In [17]:
df.loc[df['PostalCode'].isin(codes_in_example)]       

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
23,M4G,East York,Leaside,43.70902,-79.36349
24,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493
27,M2H,North York,Hillcrest Village,43.80225,-79.35558
32,M1J,Scarborough,Scarborough Village,43.74446,-79.23117
50,M9L,North York,Humber Summit,43.75948,-79.55707
54,M4M,East Toronto,Studio District,43.66213,-79.33497
71,M1R,Scarborough,"Wexford, Maryvale",43.75075,-79.30054


***Assgnment 2 ENDS HERE***


---------------------------------


* ## Assignment 3
Explore and cluster the neighborhoods in Toronto. You decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. generate maps to visualize neighborhoods and clusters.

In [24]:
!pip install geopy
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium
import numpy as np

     |████████████████████████████████| 112kB 14.0MB/s eta 0:00:01


In [25]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent = 'foursquare_agent')
location = geolocator.geocode(address)
mymap = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

### A map of all the neighborhoods in toronto (not clustered yet)

In [26]:
mask = df['Borough'].str.contains(r'Toronto', na=True)              #keeping only entries of the table that contain "Toronto" in borough section as instructed
df_tor = df[mask].reset_index()

for lat, lon, lbl in zip(df_tor.Latitude, df_tor.Longitude, df_tor.Neighbourhood):
    label = folium.Popup(lbl, parse_html=True)
    folium.CircleMarker([lat,lon], radius = 7, color = 'red', popup = label, fill = True, fill_color = 'red', fill_opacity = 0.5).add_to(mymap)
mymap

In [27]:
CLIENT_ID = '4H221GSI4E14FOF4WKROCNN3CRVOAYJZIJ2ZJQ3Q5WRBBEJZ' # your Foursquare ID
CLIENT_SECRET = 'RNRQT0MROZPVEMNOCC5HKHKXGBRWOFUH3OJJMEGLCMDWS3YD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

### Get venue data for each neighborhood to use for clustering

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=200):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
toronto_nearby_venues = getNearbyVenues(df_tor['Neighbourhood'], df_tor['Latitude'], df_tor['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [31]:
toronto_nearby_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65512,-79.36264,Souvlaki Express,43.655584,-79.364438,Greek Restaurant
...,...,...,...,...,...,...,...
493,"Business reply mail Processing Centre, South C...",43.64869,-79.38544,Hilton Toronto Executive Lounge,43.650363,-79.385802,Breakfast Spot
494,"Business reply mail Processing Centre, South C...",43.64869,-79.38544,Freshii,43.649273,-79.383748,Salad Place
495,"Business reply mail Processing Centre, South C...",43.64869,-79.38544,Thai Island,43.649146,-79.383798,Thai Restaurant
496,"Business reply mail Processing Centre, South C...",43.64869,-79.38544,Momofuku Noodle Bar,43.649366,-79.386217,Noodle House


In [32]:
print('There are {} unique categories.'.format(len(toronto_nearby_venues['Venue Category'].unique())))

There are 135 unique categories.


### Encoding the venue category data for use with kmeans clustering

In [33]:
# one hot encoding
tor_onehot = pd.get_dummies(toronto_nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = toronto_nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
tor_onehot['Neighborhood']
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,...,Theme Restaurant,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.076923,0.000000,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.00,0.000000,0.00,0.040000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.0,0.000000,0.000000,0.04,0.000000,0.000000,0.000000,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.142857,0.000000,0.00,0.142857,0.000000,0.000000,0.0,0.000000
4,Central Bay Street,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000
5,Christie,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000
6,Church and Wellesley,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.066667,0.0,0.000000,0.000000,0.00,0.000000,0.066667,0.000000,0.0,0.000000
7,"Commerce Court, Victoria Hotel",0.027027,0.00,0.027027,0.00,0.000000,0.000000,0.027027,0.054054,0.000000,...,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.027027,0.0,0.000000
8,Davisville,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000
9,Davisville North,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000


### Make table with top 10 venues for each neighborhood

In [130]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [131]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Hotel,Flower Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Sandwich Place,Gym,Italian Restaurant,Falafel Restaurant,Lounge,Pet Store,Japanese Restaurant,Sports Bar,Convenience Store
2,"Business reply mail Processing Centre, South C...",Café,Hotel,Steakhouse,Japanese Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Breakfast Spot,Juice Bar,Fast Food Restaurant,Lounge
3,"CN Tower, King and Spadina, Railway Lands, Har...",Ramen Restaurant,Café,Veterinarian,Coffee Shop,Train Station,Caribbean Restaurant,Restaurant,Diner,Ethiopian Restaurant,Dumpling Restaurant
4,Central Bay Street,Sushi Restaurant,Japanese Restaurant,Bubble Tea Shop,Hotel,Dessert Shop,Modern European Restaurant,Coffee Shop,Miscellaneous Shop,Poke Place,Chinese Restaurant


In [132]:
# set number of clusters
kclusters = 3
tor_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [133]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = df_tor

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
tor_merged.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)
tor_merged.drop("index",1,inplace=True)
tor_merged.drop("PostalCode",1 ,inplace=True)

tor_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,0.0,Breakfast Spot,Yoga Studio,Bakery,Coffee Shop,Sandwich Place,Pharmacy,Greek Restaurant,Gift Shop,Gluten-free Restaurant,Cupcake Shop
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,1.0,College Cafeteria,Park,Yoga Studio,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Diner
2,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,0.0,Coffee Shop,Café,College Rec Center,Burrito Place,Lake,Ethiopian Restaurant,Middle Eastern Restaurant,Other Great Outdoors,Pizza Place,Pub
3,Downtown Toronto,St. James Town,43.65215,-79.37587,0.0,Japanese Restaurant,Restaurant,Gastropub,Hotel,Camera Store,Coffee Shop,Italian Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant
4,East Toronto,The Beaches,43.67709,-79.29547,2.0,Trail,Discount Store,Fish & Chips Shop,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Dog Run,Yoga Studio,Flower Shop


### Create folium map with clusters

In [134]:
tor_merged = tor_merged.dropna(axis = 0)

In [135]:
tor_merged['Cluster Labels'] = tor_merged['Cluster Labels'].astype(int)

In [136]:
tor_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,0,Breakfast Spot,Yoga Studio,Bakery,Coffee Shop,Sandwich Place,Pharmacy,Greek Restaurant,Gift Shop,Gluten-free Restaurant,Cupcake Shop
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,1,College Cafeteria,Park,Yoga Studio,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Diner
2,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,0,Coffee Shop,Café,College Rec Center,Burrito Place,Lake,Ethiopian Restaurant,Middle Eastern Restaurant,Other Great Outdoors,Pizza Place,Pub
3,Downtown Toronto,St. James Town,43.65215,-79.37587,0,Japanese Restaurant,Restaurant,Gastropub,Hotel,Camera Store,Coffee Shop,Italian Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant
4,East Toronto,The Beaches,43.67709,-79.29547,2,Trail,Discount Store,Fish & Chips Shop,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Dog Run,Yoga Studio,Flower Shop
5,Downtown Toronto,Berczy Park,43.64536,-79.37306,0,Hotel,Flower Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
6,Downtown Toronto,Central Bay Street,43.65609,-79.38493,0,Sushi Restaurant,Japanese Restaurant,Bubble Tea Shop,Hotel,Dessert Shop,Modern European Restaurant,Coffee Shop,Miscellaneous Shop,Poke Place,Chinese Restaurant
7,Downtown Toronto,Christie,43.66869,-79.42071,0,Flower Shop,Grocery Store,Café,Nightclub,Dog Run,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Discount Store
8,Downtown Toronto,"Richmond, Adelaide, King",43.64970,-79.38258,0,Coffee Shop,Salad Place,Seafood Restaurant,Thai Restaurant,American Restaurant,Building,General Travel,Restaurant,Juice Bar,Japanese Restaurant
9,West Toronto,"Dufferin, Dovercourt Village",43.66505,-79.43891,0,Convenience Store,Brazilian Restaurant,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Discount Store,Flower Shop


In [137]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
           
map_clusters

***Assgnment 3 ENDS HERE***


---------------------------------